In [1]:
import pandas as pd
import fiona
from rdflib import Graph, Namespace, URIRef, XSD, Literal

Dataset of population of Austrian municipalities: 

https://regionalatlas.statistikportal.de/#

In [6]:
# import population data Austria
population_data = pd.read_csv("/mnt/data/processed/tmp/bevölkerung_gemeinden_deutschland.csv", skiprows=0, delimiter=",", encoding="utf-8")

In [8]:
population_data.head()

,id,gen,jahr,pop_2021
0,465881,Flensburg,2021,79131.0
1,465882,Kiel,2021,233102.0
2,465883,Lübeck,2021,213739.0
3,465884,Neumünster,2021,79455.0
4,465885,Brunsbüttel,2021,9468.0


In [10]:
# load gadm dataset 
path_to_gadm_graph = "/mnt/data/processed/RDF/GADM/gadm_germany.ttl"

gadm = Namespace("http://example.com/ontologies/gadm#")

graph = Graph()
graph.bind("gadm", gadm)

graph.parse(path_to_gadm_graph, format='ttl')

<Graph identifier=N6171308966e6438fafcf60f05981c5fc (<class 'rdflib.graph.Graph'>)>

In [11]:
ent_hasPop = URIRef(gadm["hasPopulation2021"])

In [13]:
query = """
SELECT ?subject ?name
WHERE {
  ?subject gadm:hasNationalLevel 3 ;
           gadm:hasName ?name .
}
"""

# Execute the query and iterate over the results
for row in graph.query(query):
    admin_ent, name = row
    try:
        pop = population_data[population_data.gen == name.strip()]["pop_2021"].values[0]
        graph.add((admin_ent, ent_hasPop, Literal(pop, datatype=XSD.integer))) 
        #print(name, pop)
    except:
        pass

In [14]:
graph.serialize("/mnt/data/processed/tmp/gadm_germany_population.ttl")

<Graph identifier=N6171308966e6438fafcf60f05981c5fc (<class 'rdflib.graph.Graph'>)>